<h1 align=Left><font size = 5>Segmenting and Clustering Neighborhoods in San Francisco - Housing and Community Development</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Download and Explore Dataset</a>

2.  <a href="#item2">Explore Neighborhoods in New York City</a>

3.  <a href="#item3">Analyze Each Neighborhood</a>

4.  <a href="#item4">Cluster Neighborhoods</a>

5.  <a href="#item5">Examine Clusters</a>  
    </font>
    </div>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
!pip install folium
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import requests # library to handle requests

print('Libraries imported.')

     |████████████████████████████████| 93 kB 4.0 MB/s  eta 0:00:01
Libraries imported.


In [2]:
!wget -q -O 'sf_housing_data.json' https://data.sfgov.org/resource/9rdx-httc.geojson
print('Data downloaded!')

Data downloaded!


In [3]:
with open('sf_housing_data.json') as json_data:
    sf_data = json.load(json_data)

sf_data

neighborhoods_data = sf_data['features']
#neighborhoods_data[0]
# define the dataframe columns
column_names = [ 'Neighborhood', 'Project_Name', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

for data in neighborhoods_data:
   # borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['neighborhood']
    project_name = data['properties']['project_name']
        
   # neighborhood_latlon = data['properties']['latitude']
    neighborhood_lat = data['properties']['latitude']
    neighborhood_lon = data['properties']['longitude']
    
    neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Project_Name': project_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods
#neighborhoods.describe()
#neighborhoods.head(10)

,Neighborhood,Project_Name,Latitude,Longitude
0,Confidential,SafeHouse,None,None
1,Tenderloin,William Penn Hotel,37.78,-122.4102631
2,Noe Valley,60 28th Street,37.75,-122.4234398
3,Bayview Hunters Point,Geraldine Johnson Manor,37.73,-122.3931198
4,South of Market,1028 Howard,37.78,-122.4079208
...,...,...,...,...
371,Tenderloin,990 Polk,37.79,-122.4193802
372,Lone Mountain/USF,The Coronet,37.78,-122.4579163
373,Mission,3353 26th Street - Small Sites,37.75,-122.4171635
374,Mission,Mission Hotel,37.76,-122.4178848


In [4]:
neighborhoods['Latitude'].replace("None", np.nan, inplace = True)
neighborhoods.dropna(subset=["Latitude"], axis=0, inplace=True)
# reset index, because we droped two rows
neighborhoods.reset_index(drop=True, inplace=True)
neighborhoods
neighborhoods.describe()

,Neighborhood,Project_Name,Latitude,Longitude
count,371,371,371,371
unique,36,371,13,356
top,Tenderloin,Ferguson Place,37.78,-122.3736725
freq,70,1,147,2


In [5]:
print('The dataframe has {} neighborhoods and {} different projects.'.format(
        len(neighborhoods['Neighborhood'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 36 neighborhoods and 371 different projects.


In [6]:
address = 'San Francisco, CA'

geolocator = Nominatim(user_agent="sf_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


In [7]:
# create map of New York using latitude and longitude values
map_sf = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, project_name in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood'], neighborhoods['Project_Name']):
    label = '{}, {}'.format(project_name, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)  
    
map_sf

In [8]:
neighborhoods['Neighborhood'].unique()

array(['Tenderloin', 'Noe Valley', 'Bayview Hunters Point',
       'South of Market', 'Mission', 'Lone Mountain/USF',
       'Western Addition', 'Japantown', 'Bernal Heights', 'Chinatown',
       'Hayes Valley', 'Potrero Hill', 'Treasure Island', 'North Beach',
       'Financial District/South Beach', 'Outer Richmond', 'Glen Park',
       'Haight Ashbury', 'Nob Hill', 'Mission Bay', 'Castro/Upper Market',
       'Portola', 'Russian Hill', 'Pacific Heights', 'West of Twin Peaks',
       'Twin Peaks', 'Oceanview/Merced/Ingleside', 'Outer Mission',
       'Presidio', 'McLaren Park', 'SOUTH OF MARKET', 'Visitacion Valley',
       'Excelsior', 'Inner Richmond', 'Marina', 'Richmond'], dtype=object)

In [9]:
neighborhood1_data = neighborhoods[neighborhoods['Neighborhood'] == 'Tenderloin'].reset_index(drop=True)
neighborhood1_data.head(20)

,Neighborhood,Project_Name,Latitude,Longitude
0,Tenderloin,William Penn Hotel,37.78,-122.4102631
1,Tenderloin,125 Mason Street,37.78,-122.4097443
2,Tenderloin,555 Ellis Street Family Apartments,37.78,-122.4154129
3,Tenderloin,Madonna Residence,37.78,-122.4163818
4,Tenderloin,The Nathan Building,37.78,-122.4134521
5,Tenderloin,Marathon Hotel,37.78,-122.41803
6,Tenderloin,Curran House,37.78,-122.4112244
7,Tenderloin,Yosemite Apartments,37.78,-122.4155045
8,Tenderloin,Maria Manor,37.79,-122.40905
9,Tenderloin,Presentation Senior Community,37.78,-122.411438


In [10]:
address = 'Tenderloin, CA'

geolocator = Nominatim(user_agent="sf_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tenderloin are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tenderloin are 37.7842493, -122.4139933.


In [11]:
# create map of Neighborhood using latitude and longitude values
neighborhood1_map = folium.Map(location=[latitude, longitude], zoom_start=16)

# add markers to map
for lat, lng, label in zip(neighborhood1_data['Latitude'], neighborhood1_data['Longitude'], neighborhood1_data['Project_Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(neighborhood1_map)  
    
neighborhood1_map

In [12]:
# The code was removed by Watson Studio for sharing.

In [13]:
search_query = 'grocery'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
#url

In [14]:
import requests # library to handle requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff3b7363d675168f1ab1cf4'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4c2045cf920076b0b20fc6e9',
    'name': 'G & H Liquor & Grocery',
    'location': {'address': '201 Jones St',
     'crossStreet': 'Turk Street',
     'lat': 37.78324575324384,
     'lng': -122.41246663427593,
     'labeledLatLngs': [{'label': 'display',
       'lat': 37.78324575324384,
       'lng': -122.41246663427593}],
     'distance': 174,
     'postalCode': '94102',
     'cc': 'US',
     'city': 'San Francisco',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['201 Jones St (Turk Street)',
      'San Francisco, CA 94102']},
    'categories': [{'id': '4bf58dd8d48988d186941735',
      'name': 'Liquor Store',
      'pluralName': 'Liquor Stores',
      'shortName': 'Liquor Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_liquor_',
       'suffix': '

In [15]:
neighborhood_latitude = neighborhood1_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhood1_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhood1_data.loc[0, 'Project_Name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of William Penn Hotel are 37.78, -122.4102631.


In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
#url # display URL

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff3b7365c9afa7d85118d96'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'SoMa',
  'headerFullLocation': 'SoMa, San Francisco',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 71,
  'suggestedBounds': {'ne': {'lat': 37.7845000045, 'lng': -122.4045801810179},
   'sw': {'lat': 37.7754999955, 'lng': -122.41594601898208}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5ae925e7c0f163002cc57349',
       'name': 'Birdsong',
       'location': {'address': '1085 Mission St',
        'crossStreet': '7th',
        'lat': 37.77942453003254,
        'lng': -122.41047319701762,
        'labeledLatLn

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(50)

nearby_venues['categories'].unique()
#nearbygroc_data = nearby_venues[nearby_venues['categories'] == '%Bar%'].reset_index(drop=True)
#nearbygroc_data.query('categories.str.contains("Bar")', engine='python')
#nearby_venues.query('categories.str.contains("Bar")', engine='python')
#nearbygroc_data.head()


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


array(['Restaurant', 'Pizza Place', 'Bakery', 'Dance Studio',
       'Coffee Shop', 'Mediterranean Restaurant', 'Beer Bar',
       'Art Gallery', 'Theater', 'Breakfast Spot',
       'Vietnamese Restaurant', 'Burrito Place', 'Marijuana Dispensary',
       'Bar', 'Hotel', 'Hotel Bar', 'Food Truck', 'Design Studio',
       'Arts & Crafts Store', 'Coworking Space', 'Brewery',
       'Gym / Fitness Center', 'Music Venue', 'Plaza', 'Gym',
       'Sandwich Place', 'Deli / Bodega', 'Burger Joint', 'Cocktail Bar',
       'Japanese Restaurant', 'American Restaurant', 'Burmese Restaurant',
       'South Indian Restaurant', 'Farmers Market', 'Italian Restaurant',
       'Shoe Store', 'Luggage Store', 'Street Food Gathering',
       'Performing Arts Venue', 'Karaoke Bar', 'Mexican Restaurant',
       'Diner', 'Museum', 'Thai Restaurant', 'Café', 'Bank'], dtype=object)

In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

71 venues were returned by Foursquare.


### 2. Explore the Tenderloin, CA


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, lat, lng)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Project_Name', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
tenderloin_venues = getNearbyVenues(names=neighborhood1_data['Project_Name'],
                                   latitudes=neighborhood1_data['Latitude'],
                                   longitudes=neighborhood1_data['Longitude']
                                  )

William Penn Hotel 37.78 -122.4102631
125 Mason Street 37.78 -122.4097443
555 Ellis Street Family Apartments 37.78 -122.4154129
Madonna Residence 37.78 -122.4163818
The Nathan Building 37.78 -122.4134521
Marathon Hotel 37.78 -122.41803
Curran House 37.78 -122.4112244
Yosemite Apartments 37.78 -122.4155045
Maria Manor 37.79 -122.40905
Presentation Senior Community 37.78 -122.411438
Cadillac Hotel 37.78 -122.4139404
Tenderloin Family Housing 37.78 -122.412941
Lyric Hotel 37.78 -122.4120712
Hamlin Hotel 37.78 -122.4138641
Padre Apartments 37.78 -122.4128647
Vera Haile Senior Housing 37.78 -122.4125443
Civic Center Residence 37.78 -122.4127884
Ritz Hotel 37.78 -122.4112473
Cameo Apartments 37.78 -122.4155045
St. Claire Residence 37.79 -122.4128418
Herald Hotel Apartments 37.78 -122.4129181
Hotel Essex 37.78 -122.417511
Eddy & Taylor Family Housing (210 Taylor or 168 Eddy) 37.78 -122.4106615
Tower 737 (aka Post St Towers) 37.79 -122.414032
350 Ellis 37.79 -122.4121244
Larkin St Assisted Car

In [23]:
print(tenderloin_venues.shape)
tenderloin_venues.head()

(5830, 7)


,Project_Name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,William Penn Hotel,37.78,-122.4102631,Birdsong,37.779425,-122.410473,Restaurant
1,William Penn Hotel,37.78,-122.4102631,Square Pie Guys,37.779229,-122.410870,Pizza Place
2,William Penn Hotel,37.78,-122.4102631,Frena Bakery and Cafe,37.780500,-122.408250,Bakery
3,William Penn Hotel,37.78,-122.4102631,Alonzo King LINES Dance Center,37.780116,-122.412187,Dance Studio
4,William Penn Hotel,37.78,-122.4102631,Saint Frank,37.779519,-122.410432,Coffee Shop


In [24]:
tenderloin_venues.groupby('Project_Name').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Project_Name,,,,,,
111 Jones,93,93,93,93,93,93
125 Mason Street,66,66,66,66,66,66
149 Mason Street Apartments,66,66,66,66,66,66
205 Jones,94,94,94,94,94,94
308 Turk Street,74,74,74,74,74,74
...,...,...,...,...,...,...
Turk & Eddy Apartments,86,86,86,86,86,86
Vera Haile Senior Housing,92,92,92,92,92,92
West Hotel,65,65,65,65,65,65


In [25]:
print('There are {} uniques categories.'.format(len(tenderloin_venues['Venue Category'].unique())))

There are 155 uniques categories.


In [26]:
# one hot encoding
tendorloin_onehot = pd.get_dummies(tenderloin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tendorloin_onehot['Project_Name'] = tenderloin_venues['Project_Name'] 

# move neighborhood column to the first column
fixed_columns = [tendorloin_onehot.columns[-1]] + list(tendorloin_onehot.columns[:-1])
tendorloin_onehot = tendorloin_onehot[fixed_columns]

tendorloin_onehot.head()
tendorloin_onehot.shape

(5830, 156)

In [27]:
tenderloin_grouped = tendorloin_onehot.groupby('Project_Name').mean().reset_index()
tenderloin_grouped

,Project_Name,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bagel Shop,Bakery,Bank,...,Theater,Tiki Bar,Toy / Game Store,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,111 Jones,0.000000,0.032258,0.021505,0.010753,0.010753,0.010753,0.0,0.032258,0.010753,...,0.032258,0.0,0.0,0.0,0.0,0.021505,0.021505,0.000000,0.0,0.0
1,125 Mason Street,0.015152,0.030303,0.030303,0.000000,0.015152,0.000000,0.0,0.045455,0.015152,...,0.030303,0.0,0.0,0.0,0.0,0.000000,0.045455,0.000000,0.0,0.0
2,149 Mason Street Apartments,0.015152,0.030303,0.030303,0.000000,0.015152,0.000000,0.0,0.045455,0.015152,...,0.030303,0.0,0.0,0.0,0.0,0.000000,0.045455,0.000000,0.0,0.0
3,205 Jones,0.000000,0.031915,0.021277,0.010638,0.010638,0.010638,0.0,0.031915,0.010638,...,0.031915,0.0,0.0,0.0,0.0,0.021277,0.021277,0.000000,0.0,0.0
4,308 Turk Street,0.000000,0.013514,0.027027,0.013514,0.027027,0.013514,0.0,0.027027,0.000000,...,0.040541,0.0,0.0,0.0,0.0,0.027027,0.013514,0.013514,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Turk & Eddy Apartments,0.000000,0.046512,0.023256,0.011628,0.011628,0.011628,0.0,0.034884,0.011628,...,0.023256,0.0,0.0,0.0,0.0,0.023256,0.023256,0.000000,0.0,0.0
66,Vera Haile Senior Housing,0.000000,0.032609,0.021739,0.010870,0.010870,0.010870,0.0,0.032609,0.010870,...,0.032609,0.0,0.0,0.0,0.0,0.021739,0.021739,0.000000,0.0,0.0
67,West Hotel,0.015385,0.030769,0.030769,0.000000,0.015385,0.000000,0.0,0.046154,0.015385,...,0.030769,0.0,0.0,0.0,0.0,0.000000,0.046154,0.000000,0.0,0.0
68,William Penn Hotel,0.000000,0.042254,0.028169,0.000000,0.014085,0.000000,0.0,0.042254,0.014085,...,0.028169,0.0,0.0,0.0,0.0,0.000000,0.042254,0.000000,0.0,0.0


In [28]:
tenderloin_grouped.shape

(70, 156)

In [29]:
num_top_venues = 5

for hood in tenderloin_grouped['Project_Name']:
    print("----"+hood+"----")
    temp = tenderloin_grouped[tenderloin_grouped['Project_Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----111 Jones----
                  venue  freq
0           Coffee Shop  0.10
1  Marijuana Dispensary  0.04
2           Music Venue  0.03
3                Bakery  0.03
4                  Café  0.03


----125 Mason Street----
                   venue  freq
0            Coffee Shop  0.11
1  Vietnamese Restaurant  0.05
2                 Bakery  0.05
3     Mexican Restaurant  0.03
4                    Bar  0.03


----149 Mason Street Apartments----
                   venue  freq
0            Coffee Shop  0.11
1  Vietnamese Restaurant  0.05
2                 Bakery  0.05
3     Mexican Restaurant  0.03
4                    Bar  0.03


----205 Jones----
                  venue  freq
0           Coffee Shop  0.10
1  Marijuana Dispensary  0.04
2   American Restaurant  0.03
3           Music Venue  0.03
4                Bakery  0.03


----308 Turk Street----
            venue  freq
0     Coffee Shop  0.08
1        Beer Bar  0.05
2  Sandwich Place  0.04
3            Café  0.04
4     Music Venue  

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Project_Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Project_Name'] = tenderloin_grouped['Project_Name']

for ind in np.arange(tenderloin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tenderloin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

### 3. Cluster Analysis

In [ ]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = tenderloin_grouped.drop('Project_Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tenderloin_merged = neighborhood1_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
tenderloin_merged = tenderloin_merged.join(neighborhoods_venues_sorted.set_index('Project_Name'), on='Project_Name')

tenderloin_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tenderloin_merged['Latitude'], tenderloin_merged['Longitude'], tenderloin_merged['Neighborhood'], tenderloin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
tenderloin_merged.loc[tenderloin_merged['Cluster Labels'] == 0, tenderloin_merged.columns[[1] + list(range(5, tenderloin_merged.shape[1]))]]

### Cluster Cafe & Beer

In [ ]:
tenderloin_merged.loc[tenderloin_merged['Cluster Labels'] == 1, tenderloin_merged.columns[[1] + list(range(5, tenderloin_merged.shape[1]))]]

In [ ]:
tenderloin_merged.loc[tenderloin_merged['Cluster Labels'] == 2, tenderloin_merged.columns[[1] + list(range(5, tenderloin_merged.shape[1]))]]

### Cluster Coffee Shop

In [ ]:
tenderloin_merged.loc[tenderloin_merged['Cluster Labels'] == 3, tenderloin_merged.columns[[1] + list(range(5, tenderloin_merged.shape[1]))]]

### Cluster Grocery Store

In [ ]:
tenderloin_merged.loc[tenderloin_merged['Cluster Labels'] == 4, tenderloin_merged.columns[[1] + list(range(5, tenderloin_merged.shape[1]))]]